### setup

In [1]:
import warnings
warnings.filterwarnings("ignore")

from __future__ import division

import numpy as np
import os
from glob import glob

from PIL import Image
from copy import deepcopy

from IPython.display import clear_output

from sklearn import linear_model, datasets, neighbors
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import svm

%matplotlib inline
from scipy.misc import imread, imresize
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
sns.set_context('poster')
colors = sns.color_palette("cubehelix", 5)
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42

import pandas as pd

import scipy.stats as stats
from scipy.stats import norm
import sklearn

# from importlib import reload

In [2]:
## define paths to data
curr_dir = os.getcwd()
os.path.abspath(os.path.join(curr_dir,'..','..'))

'/home/jefan/neurosketch'

In [3]:
## root paths
curr_dir = os.getcwd()
proj_dir = os.path.abspath(os.path.join(curr_dir,'..','..')) ## use relative paths
data_dir = os.path.abspath(os.path.join(curr_dir,'..','..','data')) ## use relative paths 'D:\\data'
results_dir = os.path.join(proj_dir, 'csv')
nb_name = '2_object_evidence_during_drawing'

## add helpers to python path
import sys
if os.path.join(proj_dir, 'python') not in sys.path:
    sys.path.append(os.path.join(proj_dir, 'python'))

## module definitions
import analysis_helpers as h
reload(h)
h.data_dir = data_dir
h.path_to_recog = os.path.join(data_dir, 'features/recog')
h.path_to_draw = os.path.join(data_dir, 'features/drawing')
h.roi_list_draw = np.array(['V1Draw', 'V2Draw', 'LOCDraw', 'InsulaDraw', 'postCentralDraw',
                            'preCentralDraw', 'ParietalDraw', 'FrontalDraw', 'smgDraw'])
h.roi_list_recog = ['V1','V2','LOC','IT','fusiform','parahippo','PRC','ento','hipp']
h.roi_list_recog_formatted = np.array(['V1', 'V2', 'LOC', 'FUS','PHC','IT','ENT','PRC','HC']) 
roi_list_recog = h.roi_list_recog

### get file list

In [4]:
## get raw file list for recognition runs
path_to_recog = h.path_to_recog

RECOG_METAS = sorted([i for i in os.listdir(path_to_recog) if (i.split('.')[-1]=='csv') & (i.split('_')[2][-4:] != 'Draw')])
RECOG_FEATS = sorted([i for i in os.listdir(path_to_recog) if (i.split('.')[-1]=='npy') & (i.split('_')[1][-4:] != 'Draw')])
RECOG_SUBS = np.array([i.split('_')[0] for i in RECOG_FEATS])

recog_sub_list = np.unique(RECOG_SUBS)

def preprocess_recog(RECOG_METAS, RECOG_FEATS):
    M = [i for i in RECOG_METAS if len(i.split('.')[0].split('_'))==4]
    F = [i for i in RECOG_FEATS if len(i.split('.')[0].split('_'))==4]
    return M,F

RECOG_METAS, RECOG_FEATS = preprocess_recog(RECOG_METAS, RECOG_FEATS)

In [5]:
## get raw file list for drawing runs
path_to_draw = h.path_to_draw

DRAW_METAS = sorted([i for i in os.listdir(path_to_draw) if (i.split('.')[-1]=='csv') & (i.split('_')[2][-4:] == 'Draw')])
DRAW_FEATS = sorted([i for i in os.listdir(path_to_draw) if (i.split('.')[-1]=='npy') & (i.split('_')[1][-4:] == 'Draw')])
DRAW_SUBS = np.array([i.split('_')[0] for i in DRAW_FEATS])
draw_sub_list = np.unique(DRAW_SUBS)

In [6]:
## get subject ID's that have complete datasets from all phases of experiment
sub_list = np.intersect1d(recog_sub_list,draw_sub_list)
#print('Number of subs: {}'.format(len(sub_list)))

In [7]:
## filter file list so only contains the sessions that have full datasets
def extract_good_sessions(DRAW_METAS,DRAW_FEATS,RECOG_METAS,RECOG_FEATS):
    _DRAW_METAS = [i for i in DRAW_METAS if i.split('_')[1] in sub_list]
    _DRAW_FEATS = [i for i in DRAW_FEATS if i.split('_')[0] in sub_list]
    _RECOG_METAS = [i for i in RECOG_METAS if i.split('_')[1] in sub_list]
    _RECOG_FEATS = [i for i in RECOG_FEATS if i.split('_')[0] in sub_list]
    return _DRAW_METAS, _DRAW_FEATS, _RECOG_METAS, _RECOG_FEATS

DRAW_METAS,DRAW_FEATS,RECOG_METAS,RECOG_FEATS =  \
extract_good_sessions(DRAW_METAS,DRAW_FEATS,RECOG_METAS,RECOG_FEATS)

RECOG_SUBS = np.array([i.split('_')[0]+'_neurosketch' for i in RECOG_FEATS])
RECOG_ROIS = np.array([i.split('_')[1] for i in RECOG_FEATS])

DRAW_SUBS = np.array([i.split('_')[0]+'_neurosketch' for i in DRAW_FEATS])
DRAW_ROIS = np.array([i.split('_')[1] for i in DRAW_FEATS])

In [8]:
print 'We have data from {} subjects.'.format(len(sub_list))

We have data from 31 subjects.


### How well do we do at classifying the target during production runs when we train on initial recognition patterns only?

In [11]:
reload(h)
version = '4way'
logged = True

really_run = 1
if really_run:

    ALLDM, Acc = h.make_drawing_predictions(sub_list,roi_list_recog,
                                            version=version,logged=logged)
    ## save out Acc
    np.save(os.path.join(results_dir,'decoding_object_accuracy_production.npy'),Acc)

Now analyzing hipp ...


### How well do we do at classifying the target during the pre/post recognition runs, trained on initial recognition data?

In [10]:
reload(h)
logged = True
version = '4way'

really_run = 1
if really_run:

    ALLPRE, Accpre = h.make_prepostrecog_predictions(sub_list,roi_list_recog,test_phase='pre',logged=logged)
    ALLPOST, Accpost = h.make_prepostrecog_predictions(sub_list,roi_list_recog,test_phase='post',logged=logged)

    ## add raw probs to ALLPRE and ALLPOST dataframes
    append_raw_dict = dict(zip(['t1_prob','t2_prob','c1_prob','c2_prob'],['t1_prob_raw','t2_prob_raw','c1_prob_raw','c2_prob_raw']))
    for key, value in append_raw_dict.iteritems():
        ALLPRE[value] = ALLPRE[key].apply(lambda x: np.exp(x))
        ALLPOST[value] = ALLPOST[key].apply(lambda x: np.exp(x))

    ## save out dataframe
    ALLPRE.to_csv(os.path.join(results_dir,'logistic_timeseries_{}_neural_{}_{}.csv'.format('pre',version,'logged')),index=False) ## train recog, test drawing run    
    ALLPOST.to_csv(os.path.join(results_dir,'logistic_timeseries_{}_neural_{}_{}.csv'.format('post',version,'logged')),index=False) ## train recog, test drawing run    
    ## save out Acc
    np.save(os.path.join(results_dir,'decoding_object_accuracy_recog_pre.npy'),Accpre)
    np.save(os.path.join(results_dir,'decoding_object_accuracy_recog_post.npy'),Accpost)
    Accdiff = np.array(Accpost) - np.array(Accpre)
    np.save(os.path.join(results_dir,'decoding_object_accuracy_recog_diff.npy'),Accdiff)
    print 'Done!'    

Done!


In [ ]:
## helper function to plot test accuracy on pre recognition runs; trained on initial recognition runs

def plot_recog_decoding(test_phase='pre'):
    from matplotlib import rc
    hfont = {'fontname':'Helvetica'}
    Acc = np.load(os.path.join(results_dir,'decoding_object_accuracy_recog_{}.npy'.format(test_phase)))
    Acc = np.array(Acc)
    x = pd.DataFrame(Acc.transpose())
    x.columns = roi_list_recog    
    sns.set_context('talk')
    fig = plt.figure(figsize=(8,8))
    fig = sns.barplot(data=x,palette='husl',ci=95,zorder=1)
    x2 = x
    if test_phase=='diff':
        plt.ylim(-0.25,0.25)
    else:
        plt.ylim(0,0.5)
        
    ## melt into longform dataframe to be able to make stripplot for draw decoding accuracy 
    ## for each subject
    x = x.assign(sub=sub_list)
    xl = pd.melt(x, 
                id_vars=['sub'], 
                var_name='roi',
                value_vars=roi_list_recog, 
                value_name='draw_acc')        
    fig2 = sns.swarmplot(data=xl,x="roi", 
                         y="draw_acc", 
                         #color=(0.25,0.25,0.25),
                         palette='husl',
                         dodge=True,
    #                      jitter=0.12,
                         alpha=0.6,    
                         size=6,
                         edgecolor=(0.1,0.1,0.1),
                         linewidth=1,
                         zorder=2)    

    fig3 = sns.barplot(data=x2,palette='husl',ci=95,alpha=0.1,zorder=3)
    plt.xlabel('')
    plt.title(test_phase)

    chance_dict = {'4way':0.25,'3way':0.33,'2way':0.5}
    plt.axhline(chance_dict[version],linestyle=':',color='k')
    plt.ylabel('object decoding accuracy',fontsize=20, **hfont)

    fig.set_xticklabels(h.roi_list_recog_formatted,fontsize=16, **hfont)
    out_path = os.path.join(proj_dir,'plots/{}/{}_decoding_accuracy.pdf'.format(nb_name,test_phase))
    plt.savefig(out_path,format='pdf')
    
# plot_recog_decoding(test_phase='pre')
# plot_recog_decoding(test_phase='post')    